In [6]:
import numpy
import copy
import scipy.stats as st
import math
from bqplot import LinearScale, OrdinalColorScale, ColorAxis, Axis, Lines, Figure

from ipywidgets import HBox, VBox, SelectionSlider, ToggleButton, jslink, jsdlink, Layout, IntText

In [7]:
%%html
<style>
.MathJax {float:left !important;}
.js-plot-link-container { display:none !important; }
</style>

In [16]:
import numpy
import copy

from bqplot import LinearScale, Axis, Figure
from ipywidgets import HBox, VBox, SelectionSlider, ToggleButton, jslink, Layout, Label
from IPython.display import HTML
from inspect import isclass

import ipywidgets.embed

class bqExportablePlot(object):
    hidden = Layout(visibility='hidden')
    
    initScript = '''
    <script>
        /* globals Jupyter:false */
        var bqExportable = {};

        // "click" the phantom buttons to make one particular line visible, and turn the
        // rest invisible.
        bqExportable.toggleTaggedPlots = function (name, outputRoot) {
            var readouts = outputRoot.querySelectorAll('.widget-slider > .widget-readout');

            var tagParts = [name];
            for (var i=0; i<readouts.length; i++) {
                tagParts.push(readouts[i].textContent);
            }

            var tag = tagParts.join('/');
            var buttons = outputRoot.querySelectorAll('.widget-toggle-button');

            for (var j=0; j<buttons.length; j++) {
                if (buttons[j].textContent == tag) {
                    if (buttons[j].className.indexOf('mod-active') == -1) {
                        buttons[j].click();
                    }
                }
                else if (buttons[j].className.indexOf('mod-active') > -1) {
                    buttons[j].click();
                }
            }
        };
        
        // find the output root node via the name of a plot
        bqExportable.getRootNode = function (name) {
            var referenceText = name + '/Reference';
            var labels = document.querySelectorAll('.widget-label');
            for (var i=0; i<labels.length; i++) {
                if (labels[i].textContent == referenceText) {
                    return bqExportable.findOutputRoot(labels[i]);
                }
            }

            return null;
        };
        
        // move up the DOM tree, from a widget, to find the root of the
        // output area
        bqExportable.findOutputRoot = function (target) {
            var el = target;
            while (el !== null) {
                if (el.className.indexOf('widget-subarea') > -1) {
                    return el;
                }

                if (el.className.indexOf('output_subarea') > -1) {
                    return el;
                }

                el = el.parentElement;
            }

            return null;
        };

        bqExportable.collector = function (info) {
            var totalReadouts = 0;
            var matchedReadouts = {};

            for (var name in info) {
                if (info.hasOwnProperty(name)) {
                    totalReadouts += info[name];
                    matchedReadouts[name] = [];
                }
            }

            var $self = {
                'info': info,
                'foundReadouts': [],
                'foundRoots': {},
                'matchedReadouts': matchedReadouts,
                'totalReadouts': totalReadouts
            };

            // we're finished collecting nodes when we've found all the reference nodes
            // *and* have found all needed readouts.
            $self.checkFinish = function () {
                if ($self.foundReadouts.length != $self.totalReadouts) {
                    return false;
                }

                if (Object.keys($self.foundRoots).length != Object.keys($self.info).length) {
                    return false;
                }

                return true;
            };

            // mark a node for collection if it is a slider readout or a reference
            $self.checkNode = function (m) {
                if (typeof(m.className) == 'undefined')  {
                    return;
                }

                if (typeof(m.className) != 'string')  {
                    return;
                }

                if (m.className.indexOf('widget-slider') > -1) {
                    // believe it or not, we can't use a query selector here,
                    // I guess because the observer sees the mutation before
                    // the node is actually fully connected to the DOM or
                    // something?
                    var readout = m.children[2];
                    $self.foundReadouts.push(readout);
                    return;
                }

                if (m.className.indexOf('widget-toggle-button') > -1) {
                    if (m.textContent.indexOf(name + '/') === 0) {
                        $self.foundRoots[name] = bqExportable.findOutputRoot(m);
                    }
                }
            };
            
            // match readouts to a plot name
            $self.allocateReadouts = function () {
                for (var i=0; i<$self.foundReadouts.length; i++) {
                    var readout = $self.foundReadouts[i];
                    var outputRoot = bqExportable.findOutputRoot(readout);

                    for (var name in $self.foundRoots) {
                        if ($self.foundRoots.hasOwnProperty(name)) {
                            if (outputRoot == $self.foundRoots[name]) {
                                $self.matchedReadouts[name].push(readout);
                            }
                        }
                    }
                }
            };

            $self.installHooks = function () {
                for (var name in $self.matchedReadouts) {
                    if ($self.matchedReadouts.hasOwnProperty(name)) {
                        $self.initializeSliders(name);
                    }
                }
            };

            $self.initializeSliders = function (name) {
                var outputRoot = $self.foundRoots[name];
                var readouts = $self.matchedReadouts[name];

                for (var i=0; i<readouts.length; i++) {
                    $self.linkSlider(name, readouts[i], outputRoot);
                }
            };

            $self.linkSlider = function (name, readout, outputRoot) {
                readout.addEventListener('DOMSubtreeModified', function (e) {
                    bqExportable.toggleTaggedPlots(name, outputRoot);
                });
            };

            return $self;
        };

        // jupyter loads widgets in an embedded html through an amd via require.js,
        // which has no easy event to hook into that tells us when our plot stuff
        // is loaded. SO, we've gotta create an observer on the dom and tally up
        // all the nodes we're interested in, after which we can finally hook our
        // readout-watchers up
        bqExportable.initializeExported = function (info) {
            var collector = new bqExportable.collector(info);

            var observer;
            observer = new MutationObserver(function (mutations) {
                for (var i=0; i<mutations.length; i++) {
                    for (var j=0; j<mutations[i].addedNodes.length; j++) {
                        collector.checkNode(mutations[i].addedNodes[j]);

                        if (collector.checkFinish() === true) {
                            observer.disconnect();
                            collector.allocateReadouts();
                            collector.installHooks();
                            return;
                        }
                    }
                }
            });

            observer.observe(
                document.getElementsByTagName('body')[0],
                { childList: true, subtree: true });
        };
        
        // if the jupyter notebook is live, widgets will be rendered
        // before our observer javascript is ever run, for some reason.
        // so we do it manually.
        bqExportable.initializeLive = function (info) {
            var name = Object.keys(info)[0];
            var collector = new bqExportable.collector({});
            collector.foundRoots[name] = bqExportable.getRootNode(name);
            collector.matchedReadouts[name] =
                collector.foundRoots[name]
                    .querySelectorAll('.widget-slider > .widget-readout');
            
            collector.installHooks();
        };
        
        bqExportable.initialize = function (info) {
            if (Jupyter) {
                bqExportable.initializeLive(info);
            }
            else {
                bqExportable.initializeExported(info);
            }
        };
    </script>
    '''
    
    if 'bqExportableUtil' not in ipywidgets.embed.snippet_template:
        ipywidgets.embed.snippet_template += initScript
    
    initLink = "<script>bqExportable.initialize({'%name%': %count%});</script>"
    
    def __init__(self):
        members = [getattr(self, s) for s in dir(self) if isclass(getattr(self, s))]
        slider_classes = [s for s in members if issubclass(s, bqExportableSlider)]
        figure_classes = [f for f in members if issubclass(f, bqExportableFigure)]

        self.sliders = []
        for slider in slider_classes:
            sliderInstance = slider()
            self.sliders.append(sliderInstance)
            
        self.combos = [c for c in self.gen_combinations(0, [])]
        
        self.defaultValue = None
        for combo in self.combos:
            if self.check_default(combo):
                self.defaultValue = combo
                break

        self.export = False
        self.figures = []
        
        for figure in figure_classes:
            figureInstance = figure(self.defaultValue, self.combos)
            self.figures.append(figureInstance)        
            self.export = self.export or figure.export

        # if our plot is to be exported, we'll make a new Mark for each slider
        # permutation and then jslink its visibility to a hidden toggle button.
        # each permutation being its own plot ensures that "embed widget" will
        # convert them all, and we can contol the toggles by external scripting.
        
        if self.export:
            self.buttons = []
            className = self.__class__.__name__
            for b, combo in enumerate(self.combos):
                tag = className + '/' + '/'.join([str(x) for x in combo])
                visibility = self.check_default(combo)

                button = ToggleButton(
                    value=visibility,
                    description=tag,
                    layout=self.hidden
                )
                self.buttons.append(button)

                for figure in self.figures:
                    figure.link_toggler(button, b)

            # the reference label will be used by the javascript to find the
            # output root via a plot name
            self.referenceLabel = Label(
                value='%s/Reference' % className,
                layout=self.hidden)
            
        else:
            for slider in self.sliders:
                slider.observe(self.update_plot, names='value')

    def update_plot(self, change):
        slider_values = [slider.value for slider in self.sliders]
        for figure in self.figures:
            figure.update_plot(slider_values)

    def check_default(self, combo):
        for i,item in enumerate(combo):
            if self.sliders[i].defaultValue != item:
                return False
        return True

    # slickly generate all combinations of slider inputs via generators
    # (and then gather them all into a list anyways so that we can normalize it XD)
    def gen_combinations(self, i, current_list):
        l = len(self.sliders)
        
        for val in self.sliders[i].options:
            if (i+1) == l:
                copy_list = copy.copy(current_list)
                copy_list.append(val)
                yield copy_list

            else:
                current_list.append(val)
                yield from self.gen_combinations(i+1, current_list)
                current_list.pop()

    def display(self):
        widgets_to_display = [
            self.display_sliders(),
            self.display_figures(),
        ]
        
        if self.export:
            display(HTML(self.initScript))
            widgets_to_display = widgets_to_display \
                               + [HBox([*self.buttons])] \
                               + [self.referenceLabel]
        
        display(VBox([*widgets_to_display]))

        if self.export:
            ip = get_ipython()
            ip.events.register('post_run_cell', self.run_js)

    def display_sliders(self):
        return HBox([slider.slider for slider in self.sliders])

    def display_figures(self):
        return HBox([figure.figure for figure in self.figures])
        
    def run_js(self):
        run_code = self.initLink \
                       .replace('%name%', self.__class__.__name__) \
                       .replace('%count%', str(len(self.sliders)))
        display(HTML(run_code))
        
class bqExportableSlider(object):
    options = []
    defaultValue = None
    description = ''

    def __init__(self):
        self.slider = SelectionSlider(
            options=self.options,
            value=self.defaultValue,
            description=self.description
        )
        
    @property
    def value(self):
        return self.slider.value
        
    def observe(self, callback, **kwargs):
        self.slider.observe(callback, **kwargs)
    
class bqExportableFigure(object):
    export = True
    
    xscale_options = {}
    yscale_options = {}
    
    xaxis_options = {}
    yaxis_options = {'orientation': 'vertical'}

    mark_class = None
    mark_options = {'colors': ['Gray']}

    title = ''
    figure_options = {}

    xdata = []
    ydata = []
    normalize_ydata = False

    def ygen(self, *args):
        pass

    def __init__(self, defaultValue, combos):
        self.defaultValue = defaultValue
        self.combos = combos
        self.gen_ydata(combos)

        self.marks = []
        self.xscale = LinearScale(**self.xscale_options)
        self.yscale = LinearScale(**self.yscale_options)
        self.xaxis = Axis(scale=self.xscale, **self.xaxis_options)
        self.yaxis = Axis(scale=self.yscale, **self.yaxis_options)

        for yset in self.ydata:
            mark = self.mark_class(
                x=self.xdata, y=yset,
                scales={'x': self.xscale, 'y': self.yscale},
                visible=not(self.export), **self.mark_options)
            self.marks.append(mark)
    
        if self.title != '':
            self.figure_options['title'] = self.title

        self.figure = Figure(
            marks=self.marks, axes=[self.xaxis, self.yaxis],
            **self.figure_options)
        
    def gen_ydata(self, combos):
        if not self.export:
            self.ydata = [self.ygen(*self.defaultValue)]
            return
        
        self.ydata = []

        max_val = -1
        for combo in combos:
            yset = self.ygen(*combo)
            self.ydata.append(yset)
            max_val = max(max(yset), max_val)

        # when the plot is live, an unconstrainted y-scale will auto-adjust to new
        # data when mark.y is assigned to it. however, the scales for an exported
        # plot will remain fixed, unfortunately. hence, an option to normalize.
        if self.normalize_ydata:
            for y,yset in enumerate(self.ydata):
                factor = max_val / max(yset)
                normed = factor * numpy.array(yset)
                self.ydata[y] = normed

    def link_toggler(self, button, index):
        jslink((button, 'value'), (self.marks[index], 'visible'))

    def update_plot(self, slider_values):
        self.ydata = [self.ygen(*slider_values)]
        self.marks[0].y = self.ydata[0]

In [17]:
class PriorityMin(bqExportablePlot):
    class PriorityQueueSize(bqExportableSlider):
        options = [int(x) for x in numpy.logspace(2.0, 4.0, 3)]
        defaultValue = options[1]
        description = 'Priority Queue Size'

    class RequestsPerHarvest(bqExportableSlider):
        options = [int(x) for x in numpy.logspace(2.0, 5.0, 3)]
        defaultValue = options[1]
        description = 'Requests Per Harvest'

    class Figure1(bqExportableFigure):
        export = True
        title = 'PDF of Priority Queue Minimum Priority (normalized)'
        normalize_ydata = True

        xscale_options = { 'min': 0, 'max': 1 }
        yaxis_options = { 'visible': False }
        mark_class = Lines
        mark_options = { 'colors': ['Gray'] }

        num_points = 200
        xdata = numpy.linspace(0, 1, num_points)
        yz = numpy.zeros(num_points)
        yz[0] = 1

        def ygen(self, k, n):
            if n > k:
                return st.beta.pdf(self.xdata, n+1-k, k)
            else:
                return self.yz

mp = PriorityMin()
mp.display()

In [ ]:
ipywidgets.embed.snippet_template